In [1]:
#imports
import pyodbc
import pandas as pd

In [19]:
#connect to db
driver = "{ODBC Driver 17 for SQL Server}"
server = "172.20.10.149\PRODUCTION"
database = "stb_production"
username = "Neo.Tech"
password = "Password357"

conn = pyodbc.connect("DRIVER=" + driver
+ ";SERVER=" + server
+ ";DATABASE=" + database
+ ";UID=" + username
+ ";PWD=" + password )
cursor = conn.cursor()

In [57]:
filename = 'BGA Rework - 3136 Units.xlsx'
product = '4138'
pallet_col = 'Pallet 1'
stb_find_str = '\''

excel_data = pd.read_excel('BGA Rework - 3136 Units.xlsx')
# Read the values of the file in the dataframe
df = pd.DataFrame(excel_data, columns=[pallet_col])
# Print the content
# print("The content of the file is:\n", df)

In [61]:
import time

pallet_stbs = df.iterrows()

# print(df.to_numpy().flatten())
pallet_stbs_list = df.to_numpy().flatten()

print(f'{pallet_col} has {len(pallet_stbs_list)} stbs')

test_iter = 7
stb_find_str = '\''
for stb_num_short in pallet_stbs_list:
    print(f'--iter-- {test_iter}')
    if (test_iter >= 0):
        test_iter -=1
    else:
        break
    print(f'{stb_num_short}')
    time.sleep(1)
    for row in cursor.execute(f'''SELECT stb_num FROM stb_production.dbo.production_event
    WHERE stb_num LIKE \'{stb_num_short}%\''''):
        # print(f'>> {row.stb_num}')
        # stb_find_str = stb_find_str + row.stb_num + '\',\''
        stripped_stb_num = row.stb_num.strip()
        print(f'>> {stb_find_str}')
        print(f'''{"STB_NUM"}\t\t\t{"IUC_NUM"}\t\t\t{"PCB_NUM"}''')
        for innerrow in cursor.execute(f'''SELECT stb_num, cdsn_iuc, pcb_num FROM stb_production.dbo.production_event
                                        INNER JOIN
                                        device_state_dsd_{product} ON production_event.id_production_event = device_state_dsd_{product}.id_production_event
                                        WHERE        stb_num IN (\'{stripped_stb_num}\')'''):
                                        print(f'''{innerrow.stb_num}\t\t{innerrow.cdsn_iuc}\t\t{innerrow.pcb_num}''')

Pallet 1 has 784 stbs
--iter-- 7
S041153204
--iter-- 6
S040789152
--iter-- 5
S040006732
>> '
STB_NUM			IUC_NUM			PCB_NUM
S0400067325		7030203093		308473008871
--iter-- 4
S040846160
--iter-- 3
S041400465
>> '
STB_NUM			IUC_NUM			PCB_NUM
S0414004652		7042585573		308953036166
--iter-- 2
S041170035
--iter-- 1
S040864251
--iter-- 0
S040977392
>> '
STB_NUM			IUC_NUM			PCB_NUM
S0409773923		7036343833		308781016256
--iter-- -1


In [59]:
print(f' SQL Input for Like Statement {stb_find_str}')
# for row in cursor.execute(f'''SELECT stb_num FROM stb_production.dbo.production_event
# WHERE stb_num LIKE \'{input_csv_stb_num}%\''''):
#     print(row.stb_num)

# TODO: improve, merge all above stb_nums with ''
print(f'''{"STB_NUM"}\t\t\t{"IUC_NUM"}\t\t\t{"PCB_NUM"}''')
for innerrow in cursor.execute(f'''SELECT stb_num, cdsn_iuc, pcb_num FROM stb_production.dbo.production_event
                                INNER JOIN
                                device_state_dsd_{product} ON production_event.id_production_event = device_state_dsd_{product}.id_production_event
                                WHERE        stb_num IN ({stb_find_str})'''):
                                print(f'''{innerrow.stb_num}\t\t{innerrow.cdsn_iuc}\t\t{innerrow.pcb_num}''')


 SQL Input for Like Statement 'S0400067325','S0414004652','S0409773923'
STB_NUM			IUC_NUM			PCB_NUM
S0400067325		7030203093		308473008871
S0409773923		7036343833		308781016256
S0414004652		7042585573		308953036166
